In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os

project_path = "/content/drive/MyDrive/Colab Notebooks"
os.chdir(project_path)

print(os.getcwd())

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks


In [ ]:
import pandas as pd
from datetime import datetime

# Load your daily sentiment data
daily_df = pd.read_csv("daily_news_sentiment.csv")
daily_df['date'] = pd.to_datetime(daily_df['date'])

# Load earnings sentiment
earnings_df = pd.read_csv("earnings_sentiment.csv")

# Manually map earnings call dates and sentiments
# (You need to look up when each call actually happened)
earnings_map = [
    # Format: (report_date, sentiment_score)
    # Q4 2023 (reported in late Jan 2024)
    ('2024-01-24', -0.05),  # Replace with actual date and score

    # Q1 2024 (reported in late Apr 2024)
    ('2024-04-23', 0.12),   # Replace with actual date and score

    # Q2 2024 (reported in late Jul 2024)
    ('2024-07-23', -0.03),  # Replace with actual date and score

    # Q3 2024 (reported in late Oct 2024)
    ('2024-10-23', 0.08),   # Replace with actual date and score

    # Q4 2024 (reported in late Jan 2025)
    ('2025-01-29', 0.15),   # Replace with actual date and score

    # Add more as needed
]

# Convert to dataframe
earnings_events = pd.DataFrame(earnings_map, columns=['report_date', 'earnings_sentiment'])
earnings_events['report_date'] = pd.to_datetime(earnings_events['report_date'])
earnings_events = earnings_events.sort_values('report_date')

# Function to get earnings sentiment for a given date
def get_earnings_sentiment(date, earnings_events):
    """
    Returns the most recent earnings sentiment as of this date.
    If date is before first earnings call, return 0 (neutral).
    """
    # Find most recent earnings call before or on this date
    past_calls = earnings_events[earnings_events['report_date'] <= date]

    if len(past_calls) == 0:
        return 0  # No earnings data yet
    else:
        return past_calls.iloc[-1]['earnings_sentiment']

# Apply to all dates in your daily data
daily_df['earnings_sentiment'] = daily_df['date'].apply(
    lambda x: get_earnings_sentiment(x, earnings_events)
)

# Save
daily_df.to_csv("daily_sentiment_with_earnings.csv", index=False)

print("Sample of earnings sentiment over time:")
print(daily_df[['date', 'earnings_sentiment']].head(20))

In [4]:
import pandas as pd
earnings_df = pd.read_csv("earnings_sentiment.csv")

print(earnings_df)

                                    filename   call_date  avg_sentiment  \
0  TSLA_Q3_2025_Earnings_Call_Transcript.rtf  2025-10-22       0.175250   
1  TSLA_Q2_2025_Earnings_Call_Transcript.rtf  2025-07-23       0.232913   
2  TSLA_Q4_2024_Earnings_Call_Transcript.rtf  2025-01-29       0.019609   
3  TSLA_Q3_2024_Earnings_Call_Transcript.rtf  2024-10-23       0.077639   
4  TSLA_Q2_2024_Earnings_Call_Transcript.rtf  2024-07-24       0.099173   
5  TSLA_Q1_2024_Earnings_Call_Transcript.rtf  2024-04-23       0.033616   
6  TSLA_Q4_2025_Earnings_Call_Transcript.rtf  2026-01-28       0.103768   

   neg_ratio  pos_ratio  num_chunks  
0   0.033333   0.250000         120  
1   0.053571   0.330357         112  
2   0.045455   0.068182         132  
3   0.029197   0.138686         137  
4   0.024793   0.148760         121  
5   0.054054   0.101351         148  
6   0.059322   0.186441         118  
